# Rapport Challenge IMA 205


## `Introduction`

        Le but de ce challenge est de fournir un classifieur efficace pour le diagnostic de lésions cutanées, plus précisément pour des mélanomes. Le but est de séparer les lésions en deux classes : maligne et bégnine . Pour y arriver plusieurs éléments nous ont été donné :
    - une base de donnée de 900 photographies de mélanomes
    - les masques de segmentation des mélanomes
    - un fichier csv de 600 lignes contenant une classification opérée par des dermatologues sur 600 images de la  base de donnée
    - des fichiers csv contenant des informations basiques comme le nom de l'image etc.

        Dans ce rapport, j'expliquerai le cheminement que j'ai eu pour arriver à mon classifieur final, en détail-
    lant chaque étape. Je montrerai les choses qui ont réussi et celle qui n'ont pas réussi, les chose que j' ai 
    faite et celle que j'aurais pu faire. J'essaierai à chaque fois d'analyser au mieux les causes de ces succès ou échecs.
    
## `Plan du rapport`
### `Partie 1 : Mes étapes de travail préalable à l'entrainement` 
#### `1- Lecture d'articles`
#### `2- Preprocessing des images`
#### `3- Le calcul des features`

### `Partie 2 : l'apprentissage`
### `Partie 3 : Interprétation des résultats/ Améliorations possibles`



    

## `Partie 1 : Mes étapes de travail`
### `1- Lecture d'articles`

        Au début de ce challenge j'ai été amené à lire plusieurs articles pour me documenter sur le problème du diagnostic des lésions cutanées. Ces articles m'ont principalement servi pour calculer les features, les caractéristiques propre à chaque image.
        Le premier article que j'ai lu s'intitulait :  "Computerized analysis of pigmented skin lesions: A review" un article de Konstantin Korotkov et Rafael Garcia. Cet article très complet donne toutes les étapes de la segmentation des lésions et du pre-processing, jusqu'à la classification des lésions. Concernant le calcul des features cependant, cet article ne donne pas vraiment de méthode explicite, mais se contente de donner le nom des features à calculer. Pour chaque feature citée, l'auteur se réferre directement à d'autres articles, ce qui est un peu fastidieux : l'article cite 341 papiers différents! 
        Le second article que j'ai lu s'intitulait : "Performance of a dermoscopy-based computer vision system for the diagnosis of pigmented skin lesions compared with visual evaluation by experienced dermatologists" de médecins et mathématiciens norvégiens. Dans cet article, peut etre un peu moins fourni que le précédent, les auteurs ont donné de nombreuses méthodes de calcul de features très utiles lors de ce challenge.

### `2- Preprocessing des images`
        Afin de calculer les features en un temps raisonnable, j'ai réduit la taille des images de sorte que la largeur de chaque image soit réduite a 300 pixels pour chaque image. On peut retrouver cette étape dans le notebook : preprocessing.ipynb. J'y utilise la fonction cv2.resize.
        
### `3- Le calcul des features`
        Le calcul des features a sans doute été le travail le plus difficile de ce projet, car on a du manipuler beaucoup d'images à la fois, écrire des algorithmes à partir des différents articles, et gérer les cas ou le calcul n'aboutissait pas. Je vais présenter le calcul des features par catégories :

#### `Les features RGB`
    Le calcul de ces features était suggéré par beaucoup d'articles comme un des features très basiques et facile a obtenir. Les features sont les suivantes pour chaque canal : 
    - la moyenne du canal sur la zone de la lésion
    - l'écart-type du canal sur la zone de la lésion
    Une autre feature était suggérée : la variance. J'ai jugé son calcul non pertinent car on avait déja calculé l'écart-type qui est la racine de la variance. Le calcul de ces features est fait dans le notebook : featuresRGB.ipynb
    Ce sont là les 6 premières features.
#### `Les features Core/Peripherical LAB`
    Dans le calcul de ces features on s'intéresse à la variation en intensité des canaux L,A,B (un autre espace colorimetrique) entre le centre de la lésion et sa périphérie. D'après le second article plus la lésion est diffuse plus elle est potentiellement maligne. Afin de comparer le coeur et la périphérie, on doit déja calculer les masque de coeur et de périphérie. On y arrive facilement en procédant à des érosions sur le masque binaire. On continue les érosions jusqu'à etre arrivé à 70% de la surface initiale. On a donc deux masque : le masque du coeur et celui de la periphérie. Pour chaque canal L,A,B on calcule la moyenne sur chacun des masque puis on soustrait les deux nombres. Le résultat de cette opération est la feature : Int/Out L, A ou B, dans le fichier csv 'features.csv'.
    Le calcul de ces features est effectué dans le notebook 'featuresLAB.ipynb'. On crée en premier une fonction qui calcule le masque intérieur, puis on calcule les moyenne et les features dans la fonction internal_vs_outter.
    Le calcul de ces features a été plutot rapide, mais n'a pas été concluant pour l'image IM_000720 qui avait comme masque binaire une image integralement blanche. L'érosion ne fonctionnait pas sur ce masque. J'ai retiré cette image pour l'entrainement, car elle aurait entrainé des erreurs. 
    Avec ces 3 nouvelles features on arrive a 9 features.
    
#### `Les features de frontières`

    Les dermatologues ont un moyen de déterminer si une lésion est maligne ou pas. Si la frontière de la lésion est très irrégulière il y a des chances que cette lésion soit maligne. Afin d'évaluer l'irrégularité des lésions on se base sur le second article. On se base sur une méthode appelée ANOVA qui calcule une variance empirique dans une petite fenetre. Pseudo-code de l'algorithme
    - Faire la liste des pixels de frontière
    - Calculer une fenetre de taille 15,15 autour de ce pixel(attention aux effets de bords)
    - Dans cette fenêtre calculer SSE(k) et SST(k) ce qui correspond à la variance empirique dans la zone de la fenêtre appartenant à la lésion ou à sa zone complémentaire.
    - Faire le rapport de ces deux valeurs et le concatener a une liste.
    - Repeter ce procédé pour tout les pixels de frontière.
    - Etablir les trois quartiles de cette liste.
    Ce sont là trois nouvelles features qui montre l'irregularité de la frontières. On arrive à 12 features.
    Le code est dans le notebook : feature_border.ipynb. Parfois le code n'a pas été concluant pour certaines images (pas plus que 5). J'ai remplacé leur valeur par la moyenne des autres. 

#### `Les features d'assymetrie de formes de couleurs`
    Ces features permettent de determiner si une lésion est très assymetrique ou pas. Pour cela on effectue l'algorithme suivant : 
    - calcul du centre de masse du masque de segmentation X1,
    - calcul de nouveaux masques binaires obtenus par seuillage à 0.1*i*255,
    - calcul des centres de masse de chacun de ces nouveaux masques,
    - calcul de la distance euclidienne entre ces nouveaux centres et X1 et création du vecteur des distances,
    - calcul de la moyenne et de la variance du vecteur des distances,
    Ces deux dernières valeurs correspondent à deux nouvelles features. Plus la moyenne est haute plus la lésion est assymetrique. La variance indique que le centre de masse a beaucoup changé ce qui est aussi un facteur d'assymetrie.
    On arrive à 14 features.
    
#### `D'autres features d'assymétrie`
    Dans ces features on cherche à nouveau à évaluer l'assymetrie de la lésion. Pour cela on execute l'algorithme suivant:
    - calcul du centre de masse de la lésion
    - calcul de 36 images de rotation par rapport à l'image de base
    - pour chacune de ces images calcul de:
    
$$ \delta_{1} = \sum_{i} \sum_{j>0} |I_{i,j}-I_{i,-j}| $$
$$ \delta_{2} = \sum_{i>0} \sum_{j} |I_{i,j}-I_{-i,j}| $$

    - selection des delta les plus faible en moyenne,
    - ces deux delta correspondent au 2 features d'assymetrie.
    Plus les deltas sont forts, plus l'assymétrie est forte. On arrive donc à 16 features différentes.
### `Résumé du calcul des features`

    On arrive donc à un total de 16 features qui couvre un large spectre dee caractéristiques de l'image : l'assymetrie, l'irrégularité de la frontière, des observations colorimétriques, et des analyses coeur/periphérie.
    Ces features devrait nous donner de bons résultat dans la classification. Le calcul des features a été plutôt rapide, c'est plus l'implémentation des algorithmes qui m'a pris du temps, entre l'analyse de l'algorithme et le debuggage.
    Afin d'avoir de meilleurs résultats dans la partie qui suit j'ai procédé à de l'oversampling sur les données d'entrainements. Cette methode permet d.avoir autant de données dans la classes malignant que dans la classe Benign, et donc d'améliorer l'apprentissage.
    



## `Partie 2 : l'apprentissage`

    `Pour cette partie d'apprentissage, sans doute la partie avec lequel j'ai eu le plus de mal, je me suis limité à quelques algorithmes, et je n'ai pas pu tout testé faute de temps. Dans cette partie je vous présenterai les algorithmes que j'ai tenté de faire marcher avec plus ou moins de succès.
    
    Le premier algorithme que j'ai testé était l'algorithme de Decision Trees. Après avoir testé les scores de validation croisée sur différents paramètres de l'arbre je trouvais des scores qui ne correspondait baissait alors que la profondeur de l'arbre augmentait. Pourtant les scores sur Kaggle ne suivait pas cette distribution, mais plutot la distribution inverse. Hormis ce résultat étrange j'ai pu remarquer les désavantages de Decision Trees. La variance de cet estimateur est vraiment trop importante pour en faire un estiamteur fiable. D'autant que le post-processing, c'est à dire le pruning des arbres n'est pas possible sur Python(il me semble que c'est possible en R), les Decision Trees m'ont toujours donné des résultats assez différents dans la classification. J'ai obtenu un score de 0.22 avec decision trees avec une profondeur maximale de 50. J'obtenais aussi de bien meilleur résultat avec une fonction d'impureté de type entropie que Gini index.
    
    J'ai ensuite tenté le support vector machine (SVM) que j'ai croisé à plusieurs reprises dans les articles. Pour cet algorithme plusieurs choses étaient d'une importance significative : tout d'abord le choix du kernel et le choix de la constante C, qui correspond à une penalisation du bruit. J'ai donc testé plusieurs kernel : gaussien, polynomial lineaire. Plusieurs résultats sont à noter : le classifieur gaussien ne fonctionnait absoluement pas. Il classait tous les éléments des données de test dans une seule classe à chaque fois. Ce noyau était donc à éliminer, même si je n'arrive pas a vraiment m'expliquer ce resultat aussi mauvais. Le second kernel que j'ai testé était le kernel polynomial. Ce kernel ne fonctionnait pas du tout et son lancement faisait toujours bugger le kernel python. J'ai été contraint d'abandonner cette piste. Il ne me restait donc que le kernel linaire. Celui ci fonctionnait correctement. J'ai donc pu effectuer la cross_validation sur le paramètre C, ce qui donnait un parametre C optimal à 320. Avec cette méthode j'obtenais un score de 0.24. C'était un progrès par rapport à la méthode précédente mais j'ai pu visualisé que les données n'était pas véritablement linéairement séparable ce qui montrait bien que le kernel linéaire était inévitablement limité.
    
    J'ai poursuivi avec des méthodes ensemblistes. Ces méthodes sont efficaces pour réduire la variance. J'ai commencé avec la méthode ADAboost qui combine des estimateurs faibles pour donner un estimateur efficace en sortie. Malheureusement cette méthode ne marchait pas vraiment. Je n'ai pas vraiment réussi à comprendre quels estimateurs faibles la methode de sklearn utilisait donc je ne peux pas trop interpreter les mauvais résultat que j'ai eu. 
    J'ai donc essayé un autre algorithme : randomForest. Cet algorithme combine des arbres de decisions  sur des sous ensembles de tailles inférieures à la taille du dataset original. Les random forest sélectionne aussi un nombre de features inférieur au nombre de features original. Il aurait fallu que je fasse une cross validation pour trouver les bons paramètres pour les arbres de Random Forest, mais je n'ai pas eu le temps de le faire. J'ai donc pris les paramètres de l'arbre de Decision tree que j'avais calculé précédemment. Avec cet arbre j'obtenais un score de 0.29. Random Forest est souvent un très bon classifieur mais l'interprétabilité des résultats est souvent difficile sachant que les features sélectionnées sont tirées au hasard.
    Je me suis donc penché sur un autre classifieur: Bootstrap Aggregating plus communément appelé Bagging. Le principe est le meme que pour Random Forest sauf qu'ici on considère toutes les features à chaque fois pour les arbres de décisions que l'on moyennera à la fin. Je choisis de faire 10000 arbres aléatoires et de les moyenner ensuite. J'ai estimé le paramètre optimal pour la taille des sous-échantillons à prendre pour l'entrainement. J'obtiens une taille égale à 0.6*card(S). C'est ces paramètres qui m'ont donné le meilleur score sur Kaggle égal à 0.42.
    
    
    
    
    

## `Partie 3 : Interprétation des résultats/ Améliorations possibles`

### `Interprétation des résultats`
    De nombreuses fonctions donnent l'importance des features dans la classification. Il en résulte souvent la même chose : les features concernant les couleurs RGB n'ont pas de réelles importance dans la classification. Cela est assez facilement explicable. Les conditions d'éclairage lors de la prises de vue ont une forte influence sur ces paramètres. On se retrouve avec des photographies qui n'ont pas du tout la meme balance des blancs : des photos bleutées, comme des photos un peu jaunies. Evidemment tous ces paramètres font que les features de couleurs n'ont pas vraiment de signifactions précises car elles ne représente pas la même chose.
    Les features les plus importantes sont les features correspondant à l'irrégularité des frontières. On peut donc penser que c'est une caractéristique importante dans la classification et que cela pourrait être utile aux dermatologues. Ensuite viennent les features d'assymetries puis les features de comparaison du coeur avec l'extérieur.
    
### `Améliorations possibles`

    La première chose que l'on pourrait faire, c'est de faire du preprocessing beaucoup plus poussés sur les images. Je pense notamment à deux choses en particulier : l'harmonisation des balances de blancs sur toutes les images en premier lieu. La raison à cela est de donner du sens aux features RGB. En second lieu beaucoup d'images contiennent des poils ou des défauts tels que des poussières ou autre. Plusieurs algorithmes existent pour retirer ces défauts, d'ailleurs des projets d'IMA206 se penche sur ce sujet. Une autre approche de preprocessing serait de retirer ces images de la base de données. Cela permetterait de ne pas apprendre de fausses caractéristiques. Je n'ai pas eu le temps de le faire mais ça aurait pu fonctionner.
    
    La seconde chose que l'on aurait pu faire, c'est de calculer plus de features. Des features qui me paraissent importantes comme celle sur la texture de l'image, la dimension fractale de la lésion sont des choses auxquelles j'aurai pu accorder de l'importance. Un autre travail possible sur les features aurait été de retirer les features peu importantes comme les features de couleurs, car elles apportent peu à la classification et rajoutent de la complexité au modèle.
    
    Parmi les modèles d'entrainement possibles, QDA et LDA sont revenus à plusieurs reprises dans les articles. Ayant obtenu un bon score avec les méthodes ensemblistes et après avoir discuté avec mes "adversaires" je n'ai pas jugé nécessaire de tester ces approches.
    N'ayant pas bien compris en détail le fonctionnement des réseaux de neurones convolutionnels, j'ai préféré ne pas les utiliser car je me serais lancé dans quelque chose que je ne maitrisais pas. Cependant il me semble que cette méthode est très efficace pour aborder la question. Un autre aspect qui m'a freiné dans cette approche, c'est que dans le réseaux de neurones l'interprétabilité des résultats est difficile à juger, alors qu'avec les méthodes sans deep learning, l'interprétabilité est plus facile à obtenir.
    
    
    

### Précision sur le code
Si vous voulez exécutez le code il faut le faire dans un ordre précis:
- d'abord preprocessing.ipynb
- ensuite feature LAB
- ensuite toutes les features sans ordre
- enfin le code learning.ipynb